In [14]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
# I started by importing some of the essential packages

nfl = "../Data/nfl_elo_all.csv"
main_df = pd.read_csv(nfl)
main_df
# Here I imported my main elo_rating dataframe, which I labelled main_df and printed this out

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2,quality,importance,total_rating
0,26/09/1920,1920,0,NaN,RII,STP,1503.947000,1300.000000,0.824651,0.175349,...,NaN,NaN,NaN,NaN,NaN,48,0,NaN,NaN,NaN
1,03/10/1920,1920,0,NaN,CBD,PTQ,1504.688000,1300.000000,0.825267,0.174733,...,NaN,NaN,NaN,NaN,NaN,48,0,NaN,NaN,NaN
2,03/10/1920,1920,0,NaN,CHI,MUT,1368.333000,1300.000000,0.682986,0.317014,...,NaN,NaN,NaN,NaN,NaN,20,0,NaN,NaN,NaN
3,03/10/1920,1920,0,NaN,RII,MUN,1516.108000,1478.004000,0.644171,0.355829,...,NaN,NaN,NaN,NaN,NaN,45,0,NaN,NaN,NaN
4,03/10/1920,1920,0,NaN,DAY,COL,1493.002000,1504.908000,0.575819,0.424181,...,NaN,NaN,NaN,NaN,NaN,14,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17090,23/01/2022,2021,0,d,TB,LAR,1681.797158,1632.341105,0.659005,0.340995,...,379.027146,238.112249,199.786107,1629.446888,1635.265484,27,30,96.0,100.0,98.0
17091,23/01/2022,2021,0,d,KC,BUF,1699.470145,1683.332367,0.614689,0.385311,...,434.101421,308.598809,291.477231,1700.324756,1644.036577,42,36,100.0,100.0,100.0
17092,30/01/2022,2021,0,c,KC,CIN,1713.932369,1569.498961,0.769519,0.230481,...,153.972641,291.207245,186.971396,1674.943733,1585.098917,24,27,98.0,100.0,99.0
17093,30/01/2022,2021,0,c,LAR,SF,1651.615346,1621.465562,0.633608,0.366392,...,197.148060,204.657956,125.373817,1644.929010,1621.968494,20,17,96.0,100.0,98.0


In [15]:
mask = (main_df['season'] == 2020)
reg_season_mask = main_df['playoff'].isnull()   
elo_2020 = main_df[mask]   
elo_2020 = elo_2020[reg_season_mask]
elo_2020 = elo_2020.drop(['quality', 'importance', 'total_rating', 'neutral'], axis = 1)
elo_2020
#My aim in this project is to predict the regular season record for teams, so I dropped the playoff and superbowl data
#I also dropped some non essential columns

,date,season,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,...,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2
16541,10/09/2020,2020,NaN,KC,HOU,1664.847251,1527.930047,0.761756,0.238244,1676.666068,...,0.751961,0.248039,250.964371,163.113859,240.822204,192.365616,1663.567048,1485.102109,34,20
16542,13/09/2020,2020,NaN,JAX,IND,1438.502585,1482.654778,0.529966,0.470034,1457.867226,...,0.352055,0.647945,216.979032,266.985431,131.277712,166.760627,1420.368379,1489.892671,27,20
16543,13/09/2020,2020,NaN,DET,CHI,1404.568948,1524.564659,0.421509,0.578491,1391.331987,...,0.444199,0.555801,188.318387,238.512620,192.712563,150.509484,1423.035870,1541.209626,23,27
16544,13/09/2020,2020,NaN,WSH,PHI,1368.490825,1528.560907,0.366499,0.633501,1400.244413,...,0.305872,0.694128,80.105529,-84.549663,87.907130,142.452589,1418.873053,1513.205823,27,17
16545,13/09/2020,2020,NaN,MIN,GB,1571.140027,1582.458606,0.576645,0.423355,1543.920373,...,0.531452,0.468548,272.758643,487.232745,169.663822,207.789554,1519.869687,1580.152199,34,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16792,03/01/2021,2020,NaN,HOU,TEN,1405.446866,1590.293046,0.334058,0.665942,1396.663313,...,0.309814,0.690186,224.752531,241.915792,249.061903,216.955557,1371.350492,1571.696393,38,41
16793,03/01/2021,2020,NaN,DEN,OAK,1430.051399,1449.188398,0.565621,0.434379,1422.043277,...,0.532775,0.467225,216.114816,136.534184,115.047966,184.371418,1429.251280,1466.642505,31,32
16794,03/01/2021,2020,NaN,SF,SEA,1494.784619,1617.482789,0.417721,0.582279,1483.498926,...,0.390153,0.609847,172.799305,157.993876,109.845697,199.502627,1503.272522,1619.654067,23,26
16795,03/01/2021,2020,NaN,CHI,GB,1521.382026,1678.962501,0.369832,0.630168,1500.118465,...,0.339926,0.660074,235.974167,344.789454,164.839330,266.649955,1497.160456,1674.786093,16,35


In [11]:
elo_2021 = pd.read_csv("../Data/nfl_elo_latest.csv", header = 0)
#This data is from the same website, but only contains the 2021 elo_data
elo_2021.drop(['quality', 'importance', 'total_rating'], axis = 1, inplace = True)
reg_season_mask = elo_2021['playoff'].isnull()
elo_2021 = elo_2021[reg_season_mask]
elo_2021
#I repeated the processes for the 2020 elo_data on the 2021 elo_data

,date,season,neutral,playoff,team1,team2,elo1_pre,elo2_pre,elo_prob1,elo_prob2,...,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2
0,2021-09-09,2021,0,NaN,TB,DAL,1656.235930,1452.424236,0.824539,0.175461,...,0.736090,0.263910,317.128566,397.401725,223.782352,248.082791,1639.815956,1500.264612,31,29
1,2021-09-12,2021,0,NaN,IND,SEA,1561.490084,1565.751134,0.586531,0.413469,...,0.522245,0.477755,216.666972,246.652318,113.888998,192.992819,1500.541394,1580.083300,16,28
2,2021-09-12,2021,0,NaN,TEN,ARI,1553.397217,1486.436956,0.681272,0.318728,...,0.657171,0.342829,33.684565,302.017311,190.370826,225.696909,1489.651053,1528.361042,13,38
3,2021-09-12,2021,0,NaN,DET,SF,1390.645249,1490.665951,0.449771,0.550229,...,0.343427,0.656573,269.384920,175.216924,171.513676,132.495404,1355.200609,1562.414869,33,41
4,2021-09-12,2021,0,NaN,ATL,PHI,1484.493617,1426.465972,0.670004,0.329996,...,0.635255,0.364745,17.562049,382.369867,162.493070,145.813648,1418.481268,1475.793058,6,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,2022-01-09,2021,0,NaN,LAR,SF,1636.561252,1554.970394,0.699274,0.300726,...,0.655596,0.344404,54.994140,142.511404,169.067416,140.972144,1597.459409,1580.112822,24,27
268,2022-01-09,2021,0,NaN,MIA,NE,1534.901453,1593.915459,0.508614,0.491386,...,0.479691,0.520309,207.911888,156.225506,130.904079,133.263355,1546.397530,1568.228525,33,24
269,2022-01-09,2021,0,NaN,TB,CAR,1664.443435,1370.835498,0.887386,0.112614,...,0.889954,0.110046,380.686593,174.037538,254.971182,66.767113,1639.622429,1356.555640,41,17
270,2022-01-09,2021,0,NaN,ATL,NO,1422.117870,1543.612249,0.419407,0.580593,...,0.405590,0.594410,84.406741,82.938797,118.434721,88.802173,1397.080391,1578.642541,20,30


# Elo_2020 EDA

In [16]:
elo_2020.describe()

,season,elo1_pre,elo2_pre,elo_prob1,elo_prob2,elo1_post,elo2_post,qbelo1_pre,qbelo2_pre,qb1_value_pre,...,qbelo_prob1,qbelo_prob2,qb1_game_value,qb2_game_value,qb1_value_post,qb2_value_post,qbelo1_post,qbelo2_post,score1,score2
count,256.0,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,...,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000,256.000000
mean,2020.0,1506.877668,1503.553253,0.583449,0.416551,1502.858160,1507.572762,1507.830736,1507.398782,168.634752,...,0.542568,0.457432,180.770860,178.613221,169.848363,169.518611,1506.038083,1509.191434,24.816406,24.761719
std,0.0,102.209680,101.410260,0.176896,0.176896,104.280622,104.317166,96.816296,96.860519,60.926288,...,0.190760,0.190760,134.642785,140.996141,60.774286,57.620577,99.097038,100.081522,9.577009,10.148402
min,2020.0,1277.169211,1260.258714,0.171955,0.062979,1260.258714,1256.035345,1272.454637,1259.472090,0.000000,...,0.070229,0.066932,-210.655138,-215.313628,-13.569460,5.574399,1259.472090,1254.979451,0.000000,0.000000
25%,2020.0,1434.685028,1435.025201,0.454889,0.292197,1426.238406,1434.685028,1438.067378,1437.825194,125.989999,...,0.400486,0.308846,103.060474,88.225668,130.938386,128.264674,1430.500373,1440.121792,19.000000,17.000000
50%,2020.0,1509.534291,1501.547104,0.589708,0.410292,1500.695665,1504.693871,1509.122460,1510.045612,171.251819,...,0.541113,0.458887,181.086675,180.671404,173.101051,172.269047,1506.513808,1506.848359,24.500000,25.000000
75%,2020.0,1578.638773,1583.541692,0.707803,0.545111,1582.640587,1588.578397,1572.436771,1574.929520,213.996676,...,0.691154,0.599514,270.611802,275.681320,213.744357,210.675728,1578.119346,1578.436424,31.000000,31.000000
max,2020.0,1777.933310,1761.858581,0.937021,0.828045,1777.933310,1775.119369,1757.263199,1742.288173,313.828383,...,0.933068,0.929771,552.757798,600.417397,310.130678,313.828383,1757.263199,1754.511219,56.000000,49.000000


In [17]:
elo_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 256 entries, 16541 to 16796
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            256 non-null    object 
 1   season          256 non-null    int64  
 2   playoff         0 non-null      object 
 3   team1           256 non-null    object 
 4   team2           256 non-null    object 
 5   elo1_pre        256 non-null    float64
 6   elo2_pre        256 non-null    float64
 7   elo_prob1       256 non-null    float64
 8   elo_prob2       256 non-null    float64
 9   elo1_post       256 non-null    float64
 10  elo2_post       256 non-null    float64
 11  qbelo1_pre      256 non-null    float64
 12  qbelo2_pre      256 non-null    float64
 13  qb1             256 non-null    object 
 14  qb2             256 non-null    object 
 15  qb1_value_pre   256 non-null    float64
 16  qb2_value_pre   256 non-null    float64
 17  qb1_adj         256 non-null 

In [18]:
len(elo_2020)

256

In [20]:
elo_2020.shape

(256, 29)

In [26]:
print("elo_2020 team_1 unique teams: " + str(len(elo_2020.team1.unique())))
print("elo_2020 team_2 unique teams: " + str(len(elo_2020.team2.unique())))

elo_2020 team_1 unique teams: 32
elo_2020 team_2 unique teams: 32


In [28]:
elo_2020.isnull().sum()
#Playoff is expected to be null as it is the regular season

date                0
season              0
playoff           256
team1               0
team2               0
elo1_pre            0
elo2_pre            0
elo_prob1           0
elo_prob2           0
elo1_post           0
elo2_post           0
qbelo1_pre          0
qbelo2_pre          0
qb1                 0
qb2                 0
qb1_value_pre       0
qb2_value_pre       0
qb1_adj             0
qb2_adj             0
qbelo_prob1         0
qbelo_prob2         0
qb1_game_value      0
qb2_game_value      0
qb1_value_post      0
qb2_value_post      0
qbelo1_post         0
qbelo2_post         0
score1              0
score2              0
dtype: int64

In [45]:
((elo_2020.groupby('team1')['score1'].sum()) + (elo_2020.groupby('team2')['score2'].sum())).sort_values(ascending = False)
#Checking which teams scored the most points

team1
GB     509
BUF    501
TB     492
TEN    491
NO     482
KC     473
BAL    468
SEA    459
IND    451
OAK    434
MIN    430
PIT    416
ARI    410
CLE    408
MIA    404
ATL    396
DAL    395
LAC    384
HOU    384
DET    377
SF     376
LAR    372
CHI    372
CAR    350
WSH    335
PHI    334
NE     326
DEN    323
CIN    311
JAX    306
NYG    280
NYJ    243
dtype: int64

In [59]:
(((elo_2020.groupby('qb1')['qb1_value_pre'].mean()) + (elo_2020.groupby('qb2')['qb2_value_pre'].mean()))/2).sort_values(ascending = False)
#Most valuable Quaterbacks according to elo_ratings for 2020

Patrick Mahomes       277.653571
Drew Brees            249.223516
Dak Prescott          243.224347
Lamar Jackson         239.013100
Aaron Rodgers         231.384497
Russell Wilson        230.683866
Kyler Murray          230.486143
Deshaun Watson        220.564304
Josh Allen            216.378019
Ben Roethlisberger    213.536976
Ryan Tannehill        208.549474
Derek Carr            203.076160
Ryan Fitzpatrick      188.896455
Jared Goff            188.035740
Tom Brady             186.113840
Matt Ryan             181.303671
Teddy Bridgewater     177.147773
Justin Herbert        171.957378
Matthew Stafford      171.069331
Philip Rivers         170.556663
Kirk Cousins          154.446743
Cam Newton            153.279154
Mitchell Trubisky     143.550687
Alex Smith            142.415180
Nick Foles            141.703845
Gardner Minshew       137.977689
Baker Mayfield        135.197639
Carson Wentz          134.930686
Daniel Jones          132.006834
Joe Burrow            130.535879
Jimmy Garo